In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [2]:
loader = TextLoader('./Hayao_Miyazaki.txt')

chunk_size=1000: Specifies the maximum number of characters in each chunk.

chunk_overlap=4: Specifies the number of characters that will overlap between consecutive chunks. This helps ensure that information is not lost between chunks.

The HuggingFaceEmbeddings class is likely part of a library that provides embeddings, which are numerical representations of text data. 

In [3]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

Created a chunk of size 3293, which is longer than the specified 1000
Created a chunk of size 1255, which is longer than the specified 1000
Created a chunk of size 1103, which is longer than the specified 1000
Created a chunk of size 1517, which is longer than the specified 1000
Created a chunk of size 1699, which is longer than the specified 1000
Created a chunk of size 1585, which is longer than the specified 1000
Created a chunk of size 1336, which is longer than the specified 1000
Created a chunk of size 1020, which is longer than the specified 1000
Created a chunk of size 1241, which is longer than the specified 1000
Created a chunk of size 1538, which is longer than the specified 1000
Created a chunk of size 1899, which is longer than the specified 1000
Created a chunk of size 1177, which is longer than the specified 1000
Created a chunk of size 1763, which is longer than the specified 1000
Created a chunk of size 1509, which is longer than the specified 1000
Created a chunk of s

In [4]:
from langchain.vectorstores import Pinecone
import pinecone

In [6]:
pinecone_api_key = os.getenv('PINECONE')

Pinecone is a vector db to store the embeddings

In [9]:
# Initialize Pinecone client
pinecone.init(
    api_key= pinecone_api_key,
    environment='gcp-starter'
)

We store the embeddings in Pinecone in here

In [10]:
# Define Index Name
index_name = "langchain-demo"

# Checking Index
if index_name not in pinecone.list_indexes():
  # Create new Index
  pinecone.create_index(name=index_name, metric="cosine", dimension=768)
  docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)
else:
  # Link to the existing index
  docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [11]:
docsearch

In [12]:
hf_key = os.getenv('HF_KEY')

In [13]:
from langchain.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm = HuggingFaceHub(
  repo_id=model, 
  model_kwargs={"temperature": 0.8, "top_k": 50}, 
  huggingfacehub_api_token=hf_key
)

In [14]:
from langchain import PromptTemplate

template:
- Instruction
- Context Placeholder: {context}
- Question Placeholder: {question}
- Answer Prompt

The placeholders {context} and {question} in the template will be dynamically replaced with actual values provided when generating responses.

In [15]:

template = """
The Human will ask questions about Hayao Miyazaki life. 
If you don't know the answer, just say you don't know. 
Keep the answer 2-3 sentences.

Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [32]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

RunnablePassthrough(): the question input through without modification

In [33]:
rag_chain = (
{"context": docsearch.as_retriever(),  "question": RunnablePassthrough()} 
| prompt 
| llm
| StrOutputParser() 
)

In [34]:
# # Setup for the retriever, question, prompt, LLM, and output parser
# retriever = docsearch.as_retriever()

# # Create the PromptTemplate object
# prompt = PromptTemplate(
#     template=template, 
#     input_variables=["context", "question"]
# )

# # Define a function to handle the chatbot response generation
# def generate_response(user_question):
#     # Retrieve the context 
#     context = retriever.get_relevant_documents(user_question)

#     # Format the prompt using the provided context and user question
#     formatted_prompt = prompt.format(context=context, question=user_question)

#     # Generate the response using the language model
#     llm_response = llm(formatted_prompt)

#     # Parse the response to get the final string output
#     parsed_output = StrOutputParser().parse(llm_response)

#     return parsed_output


In [35]:
# # Example usage:
# user_question = input("Ask me anything: ")
# response = generate_response(user_question)
# print(response)

In [36]:
user_input = input("Ask me anything: ")
result = rag_chain.invoke(user_input)
print(result)

Ask me anything:  how is miyazaki?



The Human will ask questions about Hayao Miyazaki life. 
If you don't know the answer, just say you don't know. 
Keep the answer 2-3 sentences.

Context: []
Question: how is miyazaki?
Answer: 

I don't have real-time information about Hayao Miyazaki's current state. As of the last update, he is enjoying his retirement, spending time with his family, and continuing to work on smaller animation projects.
